# Example of EMM for machine learning reweighting using ADMM optimisation

Using the PIMA diabetes dataset we compare the original dataset with labels to the reweighted dataset with artificial labels. We begin by importing the PIMA diabetes dataset, it is already cleaned and does not have any missing values, outliers, etc. This example is also nice since all features are continuous and the labels are binary.

In [1]:
# Get raw data, convert to csv, and import
# Update directory
import sys
sys.path.insert(1, '../')
# Import libraries
from emm import *
import os.path
from get_data import get_data
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Data locations
raw_dir = "../data/raw/pima_diabetes/archive.zip"
processed_dir = "../data/processed/pima_diabetes/"

# If processed data does exist convert raw to csv
if not os.path.exists(processed_dir + "diabetes.csv"):
    get_data.RData_to_csv(raw_dir,processed_dir)
    
# Get data into dataframe
df = pd.read_csv(processed_dir + "diabetes.csv")

# Replace 0 with nan
nan_cols = ['Glucose', 'BloodPressure','SkinThickness','Insulin','BMI']
df[nan_cols]=df[nan_cols].replace({'0':np.nan,0:np.nan})

Separating features from labels, useful for training ML models and for re-weighting.

In [2]:
#train_test_splitting of the dataset
X = df.drop(columns = 'Outcome')
# Getting Predicting Value
y = df['Outcome']

First, we look at the means for each features according to outcome.

In [3]:
marginals_mean = df.groupby('Outcome').mean()
display(marginals_mean)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,110.643863,70.877339,27.235457,130.287879,30.859674,0.429734,31.190000
1,4.865672,142.319549,75.321429,33.000000,206.846154,35.406767,0.550500,37.067164


We may also want to consider standard deviation of features.

In [4]:
marginals_std = df.groupby('Outcome').std()
display(marginals_std)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.017185,24.776906,12.161223,10.026491,102.482237,6.560737,0.299085,11.667655
1,3.741239,29.599199,12.299866,10.327595,132.699898,6.614982,0.372354,10.968254


### Matching marginals

First we construct functions $F$ upon which we take expectations of under the weighted measure. Mathematically, we express this as
$$ \mathbb E[F(x)] = \sum^N_{i=1} w_i F(x_i) $$

In [5]:
features = list(X)
marginals = {}
for feature in features:
    marginals[feature] = ['mean', 'std']
marginals

{'Pregnancies': ['mean', 'std'],
 'Glucose': ['mean', 'std'],
 'BloodPressure': ['mean', 'std'],
 'SkinThickness': ['mean', 'std'],
 'Insulin': ['mean', 'std'],
 'BMI': ['mean', 'std'],
 'DiabetesPedigreeFunction': ['mean', 'std'],
 'Age': ['mean', 'std']}

Create loss tables, using least squared loss and 

In [6]:
loss_0 = []
loss_1 = []
for feature in marginals.keys():
    for fun in marginals[feature]:
        marg = getattr(df[[feature,'Outcome']].groupby('Outcome'),fun)()
        loss_0.append(LeastSquaresLoss(marg.loc[0]))
        loss_1.append(LeastSquaresLoss(marg.loc[1]))

In [7]:
#regularizer = regularizers.EntropyRegularizer(limit=None)
regularizer = regularizers.ZeroRegularizer()
w_0, out_0, sol_0 = emm(X, marginals, loss_0, regularizer,
                     verbose=False, rho=25, eps_abs=1e-6, eps_rel=1e-6)
w_1, out_1, sol_1 = emm(X, marginals, loss_1, regularizer,
                     verbose=False, rho=24, eps_abs=1e-6, eps_rel=1e-6)

In [8]:
X_0 = X.copy()
X_0["weights"] = w_0 
X_1 = X.copy()
X_1["weights"] = w_1 

# Set theoretical outcome to train on reweighted datasets
X_0['Outcome'] = 0
X_1['Outcome'] = 1

df_w = pd.concat([X_0,X_1])

In [9]:
df_valw = df_w.drop(columns=['Outcome','weights']).multiply(df_w['weights'], axis="index")
df_valw['Outcome'] = df_w['Outcome']

In [10]:
w_marginals = df_valw.groupby('Outcome').sum()

In [11]:
display(marginals_mean)
display(w_marginals)
display(abs((marginals_mean - w_marginals)))

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,110.643863,70.877339,27.235457,130.287879,30.859674,0.429734,31.190000
1,4.865672,142.319549,75.321429,33.000000,206.846154,35.406767,0.550500,37.067164


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298017,109.695776,66.736755,18.534215,61.145945,29.912458,0.429736,31.190155
1,4.865671,140.924751,71.107624,21.075932,92.606929,34.818512,0.550500,37.067161


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,1.650631e-05,0.948087,4.140584,8.701242,69.141934,0.947216,2.055115e-06,0.000155
1,9.882674e-07,1.394798,4.213805,11.924068,114.239225,0.588255,1.493453e-07,0.000003


In [12]:
std_w = pd.concat([abs(X_0[features] - marginals_mean.loc[0]),abs(X_1[features] - marginals_mean.loc[1])])
std_w = std_w.multiply(df_w['weights'], axis="index")
std_w = pd.concat([std_w, df_w['Outcome']],axis=1)

In [15]:
display(marginals_std)
display(std_w.groupby('Outcome').sum())
display(abs((marginals_std - std_w.groupby('Outcome').sum())))

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.017185,24.776906,12.161223,10.026491,102.482237,6.560737,0.299085,11.667655
1,3.741239,29.599199,12.299866,10.327595,132.699898,6.614982,0.372354,10.968254


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,2.818333,21.154495,11.308545,6.595270,42.104581,6.079100,0.280938,10.778858
1,3.770921,25.522836,11.285113,6.288167,64.877347,6.260494,0.393658,10.875014


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,0.198852,3.622411,0.852678,3.431220,60.377656,0.481637,0.018148,0.888797
1,0.029682,4.076364,1.014753,4.039428,67.822551,0.354488,0.021304,0.093239
